In [1]:
import numpy as np
from hsi_atk.Pentara.pentiga_n import Pentiga_n

In [2]:
def gen_gene(chr_ranges):
    gene = []

    for i in range(len(chr_ranges)):
        chr_r = chr_ranges[i]
        chr_v = np.random.rand() * (chr_r[1]-chr_r[0]) + chr_r[0]
        gene.append(chr_v)

    return gene


def gen_sma_gene(sma_range):
    return np.random.randint(sma_range[0], sma_range[1], 2)


def gen_chr_range(n_chr, chr_range):
    chr_ranges = []

    for i in range(n_chr):
        rang = np.random.randint(chr_range[0], chr_range[1], 2)
        chr_ranges.append(rang)

    return chr_ranges


def gen_gene_family(n_fam, sma_range, n_chr, chr_range, k_range, kk_range):
    gene_family = {}

    for i in range(n_fam):
        gene_fam = 'fam_' + str(i)
        sma = np.random.randint(sma_range[0], sma_range[1], 2)
        chr_ranges = gen_chr_range(n_chr, chr_range)
        k = np.random.rand() * (k_range[1]-k_range[0]) + k_range[0]
        kk = np.random.rand() * (kk_range[1] - kk_range[0]) + kk_range[0]
        gene = gen_gene(chr_ranges)
        gene_family[gene_fam] = {}
        gene_family[gene_fam]['sma'] = sma
        gene_family[gene_fam]['gene'] = gene
        gene_family[gene_fam]['k'] = k
        gene_family[gene_fam]['kk'] = kk

    return gene_family


def gen_gene_func(gene, k, kk):
    def gene_func(x, y, z):
        return gene[0]*np.cos(gene[1]*x+gene[2]) + gene[3]*np.sin(gene[4]*x+gene[5]) + \
               gene[6]*np.cos(gene[7]*y+gene[8]) + gene[9]*np.sin(gene[10]*y+gene[11]) + \
               gene[12]*np.cos(gene[13]*z+gene[14]) + gene[15]*np.sin(gene[16]*z+gene[17]) + \
               k + z**kk
    return gene_func


def gen_k(k_range):
    return np.random.rand() * (k_range[1]-k_range[0]) + k_range[0]


def gen_pentigas(gene_family, bands=40):
    pents = {}

    for i in range(len(gene_family.keys())):
        name = "pent_" + str(i)
        fam = "fam_" + str(i)
        sma = gene_family[fam]['sma']
        gene = gene_family[fam]['gene']
        k = gene_family[fam]['k']
        kk = gene_family[fam]['kk']
        func = gen_gene_func(gene, k, kk)
        pent = Pentiga_n(name, sma, bands=bands)
        pent.set_scale_func(func)
        pents[name] = pent

    return pents

In [3]:
# Generate 5000 randomized pentiga kernels
gene_family = gen_gene_family(5000, (18, 25), 18, (0.05, 400), (100, 500), (0.5, 1.5))
pents = gen_pentigas(gene_family)
pent = pents['pent_0']
# img = pent.compose()
# print(img.shape)

(47, 41, 40)


In [4]:
# Defining label functions for pentigas - excluding categorical labels for time being
def wt_func(x):
    return (x ** 0.75) / 1000

def lp_func(x): return x ** 0.95

def palm_func(x): return x ** 0.45

def lino_func(x): return x ** 0.65

def olei_func(x): return x ** 0.55

def stea_func(x): return x ** 0.185

for pent in pents.keys():
    n_pent = pents[pent]
    n_pent.set_wt_lbl_fn(wt_func)
    n_pent.set_lp_fn(lp_func)
    n_pent.set_palm_fn(palm_func)
    n_pent.set_lino_fn(lino_func)
    n_pent.set_olei_fn(olei_func)
    n_pent.set_stea_fn(stea_func)

In [5]:
# Getting training and testing data
pent_keys = np.array(list(pents.keys()))
idx = np.random.permutation(5000)
training_idx, test_idx = idx[:4700], idx[4700:]
images = []
labels = []
for pent in pent_keys:
    n_pent = pents[pent]
    img = n_pent.compose()
    sh = img.shape
    base = np.zeros((50, 50, 40))
    s1, s2 = 25-int(sh[0]/2), 25+int(sh[0]/2)+1
    ss1, ss2 = 25-int(sh[1]/2), 25+int(sh[1]/2)+1
    base[s1:s2, ss1:ss2, :] += img
    lbl = [n_pent.gen_wt_label()]
    lbl.extend(n_pent.gen_lp_labels())
    images.append(base)
    labels.append(lbl)
    
print(images[0].shape)

(50, 50, 40)


In [6]:
# setting training and test set
images = np.stack(images, axis=-1)
images = images.swapaxes(0,3).swapaxes(1,3).swapaxes(2,3)
labels = np.array(labels)
print(images.shape)
print(labels.shape)
X_train = images[training_idx, :, :, :]
X_test = images[test_idx, :, :, :]
y_train = labels[training_idx, :]
y_test = labels[test_idx, :]

from hsi_atk.utils.hsi2color import hsi2color
wl = []
_WAVELENGTHS = [393.91, 395.982, 398.054, 400.126, 402.198, 404.27, 406.342, 408.414, 410.486, 412.558, 414.63, 416.702, 418.774, 420.846, 422.918, 424.99, 427.062, 429.134, 431.206, 433.278, 435.35, 437.422, 439.494, 441.566, 443.638, 445.71, 447.782, 449.854, 451.926, 453.998, 456.07, 458.142, 460.214, 462.286, 464.358, 466.43, 468.502, 470.574, 472.646, 474.718, 476.79, 478.862, 480.934, 483.006, 485.078, 487.15, 489.222, 491.294, 493.366, 495.438, 497.51, 499.582, 501.654, 503.726, 505.798, 507.87, 509.942, 512.014, 514.086, 516.158, 518.23, 520.302, 522.374, 524.446, 526.518, 528.59, 530.662, 532.734, 534.806, 536.878, 538.95, 541.022, 543.094, 545.166, 547.238, 549.31, 551.382, 553.454, 555.526, 557.598, 559.67, 561.742, 563.814, 565.886, 567.958, 570.03, 572.102, 574.174, 576.246, 578.318, 580.39, 582.462, 584.534, 586.606, 588.678, 590.75, 592.822, 594.894, 596.966, 599.038, 601.11, 603.182, 605.254, 607.326, 609.398, 611.47, 613.542, 615.614, 617.686, 619.758, 621.83, 623.902, 625.974, 628.046, 630.118, 632.19, 634.262, 636.334, 638.406, 640.478, 642.55, 644.622, 646.694, 648.766, 650.838, 652.91, 654.982, 657.054, 659.126, 661.198, 663.27, 665.342, 667.414, 669.486, 671.558, 673.63, 675.702, 677.774, 679.846, 681.918, 683.99, 686.062, 688.134, 690.206, 692.278, 694.35, 696.422, 698.494, 700.566, 702.638, 704.71, 706.782, 708.854, 710.926, 712.998, 715.07, 717.142, 719.214, 721.286, 723.358, 725.43, 727.502, 729.574, 731.646, 733.718, 735.79, 737.862, 739.934, 742.006, 744.078, 746.15, 748.222, 750.294, 752.366, 754.438, 756.51, 758.582, 760.654, 762.726, 764.798, 766.87, 768.942, 771.014, 773.086, 775.158, 777.23, 779.302, 781.374, 783.446, 785.518, 787.59, 789.662, 791.734, 793.806, 795.878, 797.95, 800.022, 802.094, 804.166, 806.238, 808.31, 810.382, 812.454, 814.526, 816.598, 818.67, 820.742, 822.814, 824.886, 826.958, 829.03, 831.102, 833.174, 835.246, 837.318, 839.39, 841.462, 843.534, 845.606, 847.678, 849.75, 851.822, 853.894, 855.966, 858.038, 860.11, 862.182, 864.254, 866.326, 868.398, 870.47, 872.542, 874.614, 876.686, 878.758, 880.83, 882.902, 884.974, 887.046, 889.118]
for i in range(len(_WAVELENGTHS)):
    if i%6==0:
        wl.append(_WAVELENGTHS[i])
    else:
        continue
test_rgb = []
for i in range(X_test.shape[0]):
    rgb = hsi2color(X_test[i,:,:,:], wavelengths=wl)
    test_rgb.append(rgb)
test_rgb = np.stack(test_rgb, axis=-1)

(5000, 50, 50, 40)
(5000, 6)


In [12]:
from keras import models, optimizers
from keras.layers import core, convolutional
from keras.constraints import nonneg

# build the cnn model
def build_cnn_cont(n_out=6):
    model = models.Sequential()

#     model.add(convolutional.ZeroPadding2D(padding=2))
    model.add(convolutional.SeparableConv2D(filters=64, kernel_size=5, strides=1, activation='relu', depth_multiplier=8))
    model.add(convolutional.SeparableConv2D(filters=128, kernel_size=3, strides=1, activation='relu', depth_multiplier=4))
    model.add(convolutional.MaxPooling2D(pool_size=2))
    model.add(convolutional.SeparableConv2D(filters=128, kernel_size=3, strides=1, activation='relu', depth_multiplier=2))
    model.add(convolutional.SeparableConv2D(filters=256, kernel_size=3, strides=1, activation='relu', depth_multiplier=2))
    model.add(convolutional.SeparableConv2D(filters=128, kernel_size=3, strides=1, activation='relu', depth_multiplier=2))
    model.add(convolutional.MaxPooling2D(pool_size=2))
    model.add(convolutional.SeparableConv2D(filters=128, kernel_size=3, strides=1, activation='relu', depth_multiplier=2))

    model.add(core.Flatten())
    model.add(core.Dense(768, activation='relu'))
#     model.add(core.Dropout(.5))
    model.add(core.Dense(384, activation='relu'))
#     model.add(core.Dropout(.25))
    model.add(core.Dense(192, activation='relu'))
#     model.add(core.Dense(64, activation='linear'))
    model.add(core.Dense(64, activation='relu'))
    model.add(core.Dense(32, activation='elu'))
#     model.add(core.Dense(n_out, activation='relu'))
    model.add(core.Dense(n_out, activation='selu', W_constraint=nonneg()))
    model.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-9), loss='mean_squared_error')

    return model

mdl = build_cnn_cont()

/anaconda3/envs/HSI/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="selu", kernel_constraint=<keras.con...)`


In [13]:
print(X_train.shape)
history = mdl.fit(
    x=X_train,
    y=y_train,
    batch_size=50,
    epochs=40,
#     callbacks=[WeightsLogger(root_path="/Users/tensorstrings/hsi_atk/hsi_atk/hsi_atk/data")]
)

predictions = mdl.predict(
    x=X_test,
    batch_size=50,
)

from sklearn.metrics import mean_squared_error

print(mean_squared_error(y_test[0], predictions[0]))
print(mean_squared_error(y_test[1], predictions[1]))
print(mean_squared_error(y_test[2], predictions[2]))
print(mean_squared_error(y_test[3], predictions[3]))
print(mean_squared_error(y_test[4], predictions[4]))
print(mean_squared_error(y_test[5], predictions[5]))


(4700, 50, 50, 40)
Epoch 1/40
4700/4700 [==============================] - 125s 27ms/step - loss: 36688.2343
Epoch 2/40
4700/4700 [==============================] - 121s 26ms/step - loss: 5718.7602
Epoch 3/40
4700/4700 [==============================] - 121s 26ms/step - loss: 1224.5023
Epoch 4/40
4700/4700 [==============================] - 125s 26ms/step - loss: 137.9748
Epoch 5/40
4700/4700 [==============================] - 123s 26ms/step - loss: 61.1619
Epoch 6/40
4700/4700 [==============================] - 122s 26ms/step - loss: 36.4176
Epoch 7/40
4700/4700 [==============================] - 121s 26ms/step - loss: 36.4297
Epoch 8/40
4700/4700 [==============================] - 123s 26ms/step - loss: 28.9409
Epoch 9/40
4700/4700 [==============================] - 121s 26ms/step - loss: 25.3399
Epoch 10/40
4700/4700 [==============================] - 121s 26ms/step - loss: 24.0258
Epoch 11/40
4700/4700 [==============================] - 122s 26ms/step - loss: 20.6126
Epoch 12/40
47

In [14]:
print(y_test[:3,:])
print(predictions[:3,:])

[[  2.09593116e-01   8.71787994e+02   2.47073523e+01   1.02768514e+02
    5.03898590e+01   3.73771626e+00]
 [  1.78494380e-01   7.11308107e+02   2.24375135e+01   8.94144975e+01
    4.47910593e+01   3.59253263e+00]
 [  1.93214573e-01   7.86412693e+02   2.35301084e+01   9.57711192e+01
    4.74710945e+01   3.66344636e+00]]
[[  1.38844296e-01   8.59315857e+02   2.41482296e+01   1.01129242e+02
    4.94857063e+01   3.65199852e+00]
 [  5.12359664e-03   7.14589111e+02   2.19474525e+01   8.89136658e+01
    4.42481079e+01   3.40744567e+00]
 [  1.67274788e-01   7.80727112e+02   2.33223572e+01   9.52542725e+01
    4.71459236e+01   3.65952468e+00]]
